In [1]:
import numpy as np
import sys
if "../" not in sys.path:
  sys.path.append("../") 
from lib.envs.gridworld import GridworldEnv

In [2]:
env = GridworldEnv()

In [3]:
def policy_eval_naive(policy, env, discount_factor=1.0, theta=0.00001):
    """
    Evaluate a policy given an environment and a full description of the environment's dynamics.
    
    Args:
        policy: [S, A] shaped matrix representing the policy.
        env: OpenAI env. env.P represents the transition probabilities of the environment.
            env.P[s][a] is a list of transition tuples (prob, next_state, reward, done).
        theta: We stop evaluation once our value function change is less than theta for all states.
        discount_factor: gamma discount factor.
    
    Returns:
        Vector of length env.nS representing the value function.
    """
    # Start with a random (all 0) value function
    V = np.zeros(env.nS)
    while True:
        V_next = V.copy()
        
        for state in range(env.nS):
            value = 0.0
            actions = env.P[state]
            for action, v in actions.items():
                prob, next_state, reward, done = v[0]
                value += policy[state][action] * (reward + discount_factor * prob * V[next_state])
            V_next[state] = value
        
        if np.all((V - V_next < theta)):
            break
            
        V = V_next
            
    return np.array(V)

In [6]:
def policy_eval_fast(policy, env, discount_factor=1.0, theta=0.00001):
    """
    Evaluate a policy given an environment and a full description of the environment's dynamics.
    
    Args:
        policy: [S, A] shaped matrix representing the policy.
        env: OpenAI env. env.P represents the transition probabilities of the environment.
            env.P[s][a] is a (prob, next_state, reward, done) tuple.
        theta: We stop evaluation once our value function change is less than theta for all states.
        discount_factor: gamma discount factor.
    
    Returns:
        Vector of length env.nS representing the value function.
    """
    # Start with a random (all 0) value function
    V = np.zeros(env.nS)
    
    R = np.zeros((env.nS, env.nA))
    P = np.zeros((env.nS, env.nS))
    for state in range(env.nS):
        actions = env.P[state]
        tmp = np.zeros(env.nS)
        for action, v in actions.items():
            prob, next_state, reward, _ = v[0]
            P[state][next_state] = prob
            R[state][action] = reward

    R_p = (R * policy).sum(axis=1)
    
    it = 0
    while True:
        next_V = R_p + discount_factor * P.dot(V)    
        
#         if np.all((V - next_V < theta)):
#             V = next_V
#             break
            
        it += 1
        if it == 2:
            print(next_V)
            break
            
        V = next_V.copy()
            
    return np.array(V)

In [7]:
random_policy = np.ones([env.nS, env.nA]) / env.nA
v = policy_eval_fast(random_policy, env)

[-2.25 -6.75 -8.75 -7.5  -5.25 -7.5  -8.5  -8.75 -6.5  -8.5  -7.5  -6.75
 -5.25 -7.25 -6.25 -2.75]


In [ ]:
# Test: Make sure the evaluated policy is what we expected
expected_v = np.array([0, -14, -20, -22, -14, -18, -20, -20, -20, -20, -18, -14, -22, -20, -14, 0])
np.testing.assert_array_almost_equal(v, expected_v, decimal=2)